# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
weather_df = pd.read_csv('../WeatherPy/output_data/weather_df.csv')
new_weather_df = weather_df.drop(columns=["Unnamed: 0"])
new_weather_df

,City,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Tuktoyaktuk,CA,1613072767,40,77,69.4541,-133.0374,-13.00,12.66
1,Ushuaia,AR,1613072991,75,87,-54.8000,-68.3000,50.00,11.50
2,Punta Arenas,CL,1613072991,40,47,-53.1500,-70.9167,57.20,23.02
3,Chokurdakh,RU,1613072768,33,82,70.6333,147.9167,-19.34,4.16
4,Chui,UY,1613072986,100,87,-33.6971,-53.4616,72.28,15.05
...,...,...,...,...,...,...,...,...,...
572,San Isidro,PE,1613073105,0,54,-12.1167,-77.0500,82.40,10.36
573,Malbork,PL,1613073105,97,100,54.0359,19.0266,19.00,1.07
574,Luangwa,ZM,1613073105,100,83,-15.6167,30.4167,74.52,1.63
575,Pont-à-Mousson,FR,1613073106,75,93,48.9079,6.0589,46.40,1.14


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
#lats_lngs = weather_df[['Latitude', 'Longitude']]
#humidity = weather_df["Humidity"]
locations = new_weather_df[['Latitude', 'Longitude']].astype(float)
humidity = new_weather_df["Humidity"].astype(float)


gmaps.configure(api_key=g_key)


In [43]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [82]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
temperature = (new_weather_df["Max Temp"] < 75) & (new_weather_df["Max Temp"] > 65)
wind_speed = new_weather_df["Wind Speed"] < 20
cloudiness = new_weather_df["Cloudiness"] < 20
humidity = new_weather_df["Humidity"] < 50
weather_conditions = temperature & wind_speed & cloudiness & humidity

# Use boolean indexing to filter the weather_df dataframe
perfect_weather_df = new_weather_df[weather_conditions]
perfect_weather_df = perfect_weather_df.dropna()
perfect_weather_df.head(20)

,City,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed
24,Adrar,MR,1613072994,16,16,20.5022,-10.0711,73.18,10.00
108,Sur,OM,1613072775,0,48,22.5667,59.5289,68.40,3.94
426,Atar,MR,1613073020,0,17,20.5169,-13.0499,71.67,8.79
448,Tessalit,ML,1613072858,0,16,20.1986,1.0114,73.49,7.99
461,Diffa,NE,1613073085,0,24,13.3154,12.6113,66.63,3.02
466,Potiskum,NG,1613073086,18,19,11.7091,11.0694,73.56,9.37
472,Tanout,NE,1613073087,0,17,14.9709,8.8879,73.90,10.49
511,Jiménez,MX,1613073095,1,16,28.3333,-105.4000,66.20,16.11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
hotel_df = perfect_weather_df
hotel_df['Hotel Name'] = ""
hotel_df.reset_index()
hotel_df.head(20)

,City,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
24,Adrar,MR,1613072994,16,16,20.5022,-10.0711,73.18,10.00,
108,Sur,OM,1613072775,0,48,22.5667,59.5289,68.40,3.94,
426,Atar,MR,1613073020,0,17,20.5169,-13.0499,71.67,8.79,
448,Tessalit,ML,1613072858,0,16,20.1986,1.0114,73.49,7.99,
461,Diffa,NE,1613073085,0,24,13.3154,12.6113,66.63,3.02,
466,Potiskum,NG,1613073086,18,19,11.7091,11.0694,73.56,9.37,
472,Tanout,NE,1613073087,0,17,14.9709,8.8879,73.90,10.49,
511,Jiménez,MX,1613073095,1,16,28.3333,-105.4000,66.20,16.11,


In [88]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():

    lat = row["Latitude"]
    lng = row["Longitude"]

    params["Location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_request = requests.get(base_url, params=params)

    hotel_response = hotel_request.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]

    except (KeyError, IndexError):
        print("No hotel...skipping")
#hotel_df.info()

No hotel...skipping
No hotel...skipping
No hotel...skipping
No hotel...skipping
No hotel...skipping
No hotel...skipping
No hotel...skipping
No hotel...skipping


In [90]:
target_coordinates = f"{hotel_df['Latitude'][0]},{hotel_df['Longitude'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

KeyError: 0

In [85]:
hotels = []

for city in range(len(hotel_df["City"])):

    lat = hotel_df.loc[city]["Latitude"]
    lng = hotel_df.loc[city]["Longitude"]

    city_coordinates = f"{lat},{lng}"

    params = {
        "location": city_coordinates, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except (KeyError, IndexError):
        hotels.append("Hotel not found")

# Dataframe with nearest hotel
hotel_df["Hotel Name"] = hotels
hotel_df

KeyError: 0

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
